# Sprint 02 | ETL - Dataset Collection

## Dependencies

In [1]:
!pip install spotipy --upgrade

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [3]:
### Import Libraries
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [4]:
### Set pandas settings
pd.set_option('display.max_columns', None)

## Spotify API connection

In [5]:
### Set Spotify API credentials

# Load environment variables from .env file
load_dotenv('config.env')

# Spotify API credentials
SPOTIPY_CLIENT_ID = os.environ.get('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.environ.get('SPOTIPY_CLIENT_SECRET')

# initialize Spotipy with client credentials
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID,
                                                      client_secret=SPOTIPY_CLIENT_SECRET)

spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Get Spotify API data

In [6]:
# https://everynoise.com/worldbrowser.cgi

# Replace 'your_client_id' and 'your_client_secret' with your own credentials
def get_playlist_features(playlist_ids):
    
    playlist = spotify.playlist(playlist_ids)
    playlist_id = playlist_ids
    playlist_name = playlist['name']
    playlist_description = playlist['description']
    playlist_followers = playlist['followers']['total']
    playlist_owner = playlist['owner']['display_name']
    playlist_public = playlist['public']
    playlist_snapshot_id = playlist['snapshot_id']
    
    tracks = playlist['tracks']['items']
    
    playlist_features = []
    
    for track in tracks:
        track_info = {
            'track_id': track['track']['id'],
            'track_name': track['track']['name'],
            'track_album': track['track']['album']['name'],
            'track_release_date': track['track']['album']['release_date'],
            'track_popularity': track['track']['popularity'],
            'track_duration_sec': track['track']['duration_ms'] * 0.001,
            #'track_preview_url': track['track']['preview_url'],
            #'track_cover_art_url': track['track']['album']['images'][0]['url'],
            #'track_external_url': track['track']['external_urls']['spotify'],
            #'track_album_number': track['track']['track_number'],
            'track_explicit': track['track']['explicit'],
            #'track_available_markets': track['track']['available_markets'],
            #'track_album_type': track['track']['album']['album_type'],
            #'track_added_on_playlist_date': datetime.strptime(track['added_at'], "%Y-%m-%d"),
            #'track_added_on_playlist_by': track['added_by'],
            #'artist_name': track['track']['artists'][0]['name'],
            'track_all_artists': [artist['name'] for artist in track['track']['artists']],
        }
        
        # Get audio features of the track
        track_id = track['track']['id']  # Extract track ID from the dictionary
        audio_features = spotify.audio_features([track_id])[0]  # Pass track ID as a list
        if audio_features is not None:
            track_info.update(audio_features)  # Use update instead of append to merge dictionaries
        else:
            # Handle the case where audio features are not available for the track
            # For example, you can set default values or skip appending this track_info
            pass
        
        # Getting genres and followers for the main artist of the track
        artist_info = spotify.artist(track['track']['artists'][0]['id'])
        track_info['artist_id'] = artist_info['id']
        track_info['artist_name'] = artist_info['name']
        track_info['artist_genres'] = artist_info['genres']
        track_info['artist_followers'] = artist_info['followers']['total']
        #track_info['artist_external_urls'] = artist_info['external_urls']['spotify']
        #track_info['artist_images'] = artist_info['images']['url']
        track_info['artist_popularity'] = artist_info['popularity']

        # Getting additional playlist information
        track_info['playlist_id'] = playlist_id
        track_info['playlist_name'] = playlist_name
        track_info['playlist_description'] = playlist_description
        track_info['playlist_followers'] = playlist_followers
        #track_info['playlist_owner'] = playlist_owner
        #track_info['playlist_public'] = playlist_public
        track_info['playlist_snapshot_id'] = playlist_snapshot_id
        
        playlist_features.append(track_info)
    
    return playlist_features

## Dictionary of Spotify playlists

In [48]:
### Create dictionary for Spotify Playlists
playlists = {
    # https://everynoise.com/worldbrowser.cgi
    "Top 50 - Global": "37i9dQZEVXbMDoHDwVN2tF",
    "Top 50 - USA": "37i9dQZEVXbLRQDuF5jeBp",
    "Viral 50 - Global": "37i9dQZEVXbLiRSasKsNU9",
    "Viral 50 - USA": "37i9dQZEVXbKuaTI1Z1Afx",
    "Billboard Hot 100": "6UeSakyzhiEt4NB3UAd6NQ",
    "Signed XOXO": "37i9dQZF1DX2A29LI7xHn1",
    "RapCaviar": "37i9dQZF1DX0XUsuxWHRQd",
    "Viva Latino": "37i9dQZF1DX10zKzsJ2jva",
    "Baila Reggaeton": "37i9dQZF1DWY7IeIP1cdjF",
    "Pop Rising": "37i9dQZF1DWUa8ZRTfalHk",
    "MARROW": "37i9dQZF1DXcF6B6QPhFDv",
    "All New Rock": "37i9dQZF1DWZryfp6NSvtz",
    "mint": "37i9dQZF1DX4dyzvuaRJ0n",
    "Front Page Indie": "37i9dQZF1DX2Nc3B70tvx0",
    "Chill Pop": "37i9dQZF1DX0MLFaUdXnjA",
    "Fresh Finds Country": "37i9dQZF1DWYUfsq4hxHWP",
    "RNB X": "37i9dQZF1DX4SBhb3fqCJd",
    "African Heat": "37i9dQZF1DWYkaDif7Ztbp",
    "K-Pop Rising": "37i9dQZF1DX4FcAKI5Nhzq",
    "Yalla": "37i9dQZF1DX5cO1uP1XC1g",
    "Hot Hits USA": "37i9dQZF1DX0kbJZpiYdZl",
    "Dance Rising": "37i9dQZF1DX8tZsk68tuDw",
    "Feelin' Myself": "37i9dQZF1DX6GwdWRQMQpq",
    #"Most Necessary": "37i9dQZF1DX2RxBh64BHjQ",
    "Alternative Hip-Hop": "37i9dQZF1DWTggY0yqBxES",
    "CST": "37i9dQZF1DX91gZ5XTbTPm",
    "Westside Story": "37i9dQZF1DWSvKsRPPnv5o",
    "New Joints": "37i9dQZF1DX4SrOBCjlfVi",
    "Hot Hits USA": "37i9dQZF1DX0kbJZpiYdZl",
    "sad hour": "37i9dQZF1DWSqBruwoIXkA",
    "Homegrown + Heavy": "37i9dQZF1DXdO9cVSlXgg6",
    "Fuego #052 - Regina's Fan Mixtape": "37i9dQZF1DWTwf6UlOz1Hs",
    "Fuego": "37i9dQZF1DX8sljIJzI0oo",
    "ALT NOW": "37i9dQZF1DWVqJMsgEN0F4",
    "The New Alt": "37i9dQZF1DX82GYcclJ3Ug",
    "New Noise": "37i9dQZF1DWT2jS7NwYPVI",
    "Today's Indie Rock": "37i9dQZF1DX30HHrCAl4ZG",
    "bossa pop": "37i9dQZF1DXcUY9O5yRihK",
    "Spring Pop 🌸": "37i9dQZF1DX9nkam6FKfgM",
    "New Pop Picks": "37i9dQZF1DX11otjJ7crqp",
    "Gulf & Western": "37i9dQZF1DWUPRADzDnbMq",
    "Country Rocks": "37i9dQZF1DWWH0izG4erma",
    "Country Rap": "37i9dQZF1DWXbiccytJ5L7",
    #"Cosmic Country": "37i9dQZF1DX6sc3Xn6L2DK",
    "Hot Country": "37i9dQZF1DX1lVhptIYRda",
    #"Dancehall Official": "37i9dQZF1DXan38dNVDdl4",
    "it's alt good": "37i9dQZF1DX2SK4ytI2KAZ",
    "The Scene": "37i9dQZF1DWZkHEX2YHpDV",
    "Alternative Beats": "37i9dQZF1DWXMg4uP5o3dm",
    "Soak Up The Sun": "37i9dQZF1DX6ALfRKlHn1t",
    "K-Pop ON! (온)": "37i9dQZF1DX9tPFwDMOaN1"
}

# Sort by keys and create a list of tuples
sorted_dict_by_keys = sorted(playlists.items())
# Create a new dictionary from the sorted list of tuples
sorted_playlists = dict(sorted_dict_by_keys)

## Append Spotify playlist data to `final_df`

In [8]:
### Create empty final dataframe
final_df = pd.DataFrame()

In [9]:
### Get current date and time
current_datetime = datetime.now()
month_day_year = current_datetime.strftime('%m-%d-%Y')

In [12]:
features = get_playlist_features(playlists['Top 50 - Global'])
df_1 = pd.DataFrame(features) 
df_1['playlist_date'] = month_day_year
final_df = final_df.append(df_1)
print(final_df.playlist_name.unique())
final_df

,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11,-4.263,1,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515,4,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8,-8.472,0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000,4,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10,-5.505,1,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424,4,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2,-7.643,1,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246,4,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5,-8.305,1,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639,4,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
5,3xkHsmpQCBMytMJNiDf3Ii,Beautiful Things,Fireworks & Rollerblades,2024-04-05,85,180.304,False,[Benson Boone],0.472,0.471,10,-5.692,1,0.0603,0.151000,0.000000,0.1400,0.219,105.029,audio_features,3xkHsmpQCBMytMJNiDf3Ii,spotify:track:3xkHsmpQCBMytMJNiDf3Ii,https://api.spotify.com/v1/tracks/3xkHsmpQCBMy...,https://api.spotify.com/v1/audio-analysis/3xkH...,180304,3,22wbnEMDvgVIAGdFeek6ET,Benson Boone,[singer-songwriter pop],2219737,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
6,2qSkIjg1o9h3YT9RAgYN75,Espresso,Espress

In [14]:
features = get_playlist_features(playlists['Top 50 - USA'])
df_2 = pd.DataFrame(features) 
df_2['playlist_date'] = month_day_year
final_df = final_df.append(df_2)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11,-4.263,1,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515,4,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8,-8.472,0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000,4,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10,-5.505,1,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424,4,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2,-7.643,1,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246,4,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5,-8.305,1,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639,4,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,4iZ4pt7kvcaH6Yo8UoZ4s2,Snooze,SOS,2022-12-09,90,201.800,True,[SZA],0.559,0.551,5,-7.231,1,0.1320,0.141000,0.000000,0.1100,0.392,143.008,audio_features,4iZ4pt7kvcaH6Yo8UoZ4s2,spotify:track:4iZ4pt7kvcaH6Yo8UoZ4s2,https://api.spotify.com/v1/tracks/4iZ4pt7kvcaH...,https://api.spotify.com/v1/audio-analysis/4iZ4...,201800,4,7tYKF4w9nC0nq9CsPZTHyP,SZA,"[pop, r&b, rap]",19162938,91,37i9dQZEVXbLRQDuF5jeBp,Top 50 - USA,Your daily update of the most played tracks ri...,3377106,NzY2OTA4M

In [15]:
features = get_playlist_features(playlists['Viral 50 - Global'])
df_3 = pd.DataFrame(features) 
df_3['playlist_date'] = month_day_year
final_df = final_df.append(df_3)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11,-4.263,1,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515,4,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8,-8.472,0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000,4,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10,-5.505,1,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424,4,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2,-7.643,1,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246,4,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5,-8.305,1,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639,4,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,7xjFEvchFCiAmuseCBModI,Bastardo (feat. Ñengo Flow & Jere Klein),Los Gangsters Tambien Lloran,2024-03-29,77,264.996,True,"[Pablo Chill-E, Ñengo Flow, Jere Klein]",0.517,0.746,3,-4.605,0,0.1930,0.085100,0.000000,0.0621,0.813,83.814,audio_features,7xjFEvchFCiAmuseCBModI,spotify:track:7xjFEvchFCiAmuseCBModI,https://api.spotify.com/v1/tracks/7xjFEvchFCiA...,https://api.spotify.com/v1/audio-analysis/7xjF...,264996,4,2XcZshqzPKm3iZcmt73R8D,Pablo Chill-E,"[trap chileno, urbano chilen

In [16]:
features = get_playlist_features(playlists['Viral 50 - USA'])
df_4 = pd.DataFrame(features) 
df_4['playlist_date'] = month_day_year
final_df = final_df.append(df_4)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11,-4.263,1,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515,4,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8,-8.472,0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000,4,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10,-5.505,1,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424,4,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2,-7.643,1,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246,4,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5,-8.305,1,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639,4,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,6zuPQl2RVqQ4y5nAH61CFC,II MOST WANTED,COWBOY CARTER,2024-03-27,77,208.521,False,"[Beyoncé, Miley Cyrus]",0.533,0.355,8,-6.132,1,0.0289,0.497000,0.000000,0.1740,0.256,137.986,audio_features,6zuPQl2RVqQ4y5nAH61CFC,spotify:track:6zuPQl2RVqQ4y5nAH61CFC,https://api.spotify.com/v1/tracks/6zuPQl2RVqQ4...,https://api.spotify.com/v1/audio-analysis/6zuP...,208521,4,6vWDO969PvNqNYHIOW5v0m,Beyoncé,"[pop, r&b]",37140230,92,37i9dQZEVXbKuaTI1Z1Afx,Viral 50 - USA,Your daily update of the mos

In [17]:
features = get_playlist_features(playlists['Billboard Hot 100'])
df_5 = pd.DataFrame(features) 
df_5['playlist_date'] = month_day_year
final_df = final_df.append(df_5)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11,-4.263,1,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515,4,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8,-8.472,0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000,4,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10,-5.505,1,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424,4,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2,-7.643,1,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246,4,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5,-8.305,1,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639,4,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3Vg7nPYlOrCkcV1abKbRQ1,Everyday Hustle,WE DON'T TRUST YOU,2024-03-22,81,226.946,True,"[Future, Metro Boomin, Rick Ross]",0.557,0.704,6,-5.641,0,0.3900,0.059500,0.000000,0.6760,0.378,179.494,audio_features,3Vg7nPYlOrCkcV1abKbRQ1,spotify:track:3Vg7nPYlOrCkcV1abKbRQ1,https://api.spotify.com/v1/tracks/3Vg7nPYlOrCk...,https://api.spotify.com/v1/audio-analysis/3Vg7...,226947,3,1RyvyyTE3xzB2ZywiAwp0i,Future,"[atl hip hop, hip hop, rap, southern hip hop, ...",16654071,94,6UeSakyzhiEt

In [32]:
features = get_playlist_features(playlists['RapCaviar'])
df_6 = pd.DataFrame(features) 
df_6['playlist_date'] = month_day_year
final_df = final_df.append(df_6)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.7470,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.6690,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.9340,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.9120,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.2870,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,2FI2cZGlZ1nmRt2IkjnoMi,Bless (feat. Young Thug),Bless,2024-04-05,69,162.778,True,"[Lil Wayne, Wheezy, Young Thug]",0.431,0.536,6.0,-6.049,0.0,0.2880,0.099900,0.000000,0.0944,0.3030,83.490,audio_features,2FI2cZGlZ1nmRt2IkjnoMi,spotify:track:2FI2cZGlZ1nmRt2IkjnoMi,https://api.spotify.com/v1/tracks/2FI2cZGlZ1nm...,https://api.spotify.com/v1/audio-analysis/2FI2...,162779.0,4.0,55Aa2cqylxrFIXC767Z865,Lil Wayne,"[hip hop, new orleans rap

In [31]:
features = get_playlist_features(playlists['Signed XOXO'])
df_7 = pd.DataFrame(features) 
df_7['playlist_date'] = month_day_year
final_df = final_df.append(df_7)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,50uZI42IaLYngPSHkycMuI,DM,DM,2024-03-22,56,222.992,True,"[Dee Billz, Kyle Richh, Jenn Carter]",0.734,0.678,10.0,-7.679,1.0,0.2430,0.206000,0.000000,0.1060,0.259,126.919,audio_features,50uZI42IaLYngPSHkycMuI,spotify:track:50uZI42IaLYngPSHkycMuI,https://api.spotify.com/v1/tracks/50uZI42IaLYn...,https://api.spotify.com/v1/audio-analysis/50uZ...,224882.0,4.0,2fBU7rOViK7ywwHCIYEa7h,Dee Billz,[new york drill],30581,51,37i9dQZF1DX2A29LI7xHn1,S

In [34]:
features = get_playlist_features(playlists['Viva Latino'])
df_8 = pd.DataFrame(features) 
df_8['playlist_date'] = month_day_year
final_df = final_df.append(df_8)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,3j5zNcb0aSk7dx3W3mAKzw,Pa Ti Estoy,Cosmo,2023-11-17,80,206.103,False,"[Ozuna, Anuel AA, Chris Jedi]",0.785,0.812,3.0,-3.174,1.0,0.0644,0.147000,0.000000,0.1070,0.903,162.087,audio_features,3j5zNcb0aSk7dx3W3mAKzw,spotify:track:3j5zNcb0aSk7dx3W3mAKzw,https://api.spotify.com/v1/tracks/3j5zNcb0aSk7...,https://api.spotify.com/v1/audio-analysis/3j5z...,206103.0,4.0,1i8SpTcr7yvPOmcqrbnVXY,Ozuna,"[puerto rican pop, reggaeton, trap latino, urb..

In [35]:
features = get_playlist_features(playlists['Baila Reggaeton'])
df_9 = pd.DataFrame(features) 
df_9['playlist_date'] = month_day_year
final_df = final_df.append(df_9)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,5u0YnVs3RdFQU9IIoIs9PL,Por Ti Conocí el Amor,Por Ti Conocí el Amor,2024-04-12,60,189.013,False,[Darell],0.862,0.714,7.0,-5.232,0.0,0.1430,0.442000,0.000010,0.0885,0.738,93.054,audio_features,5u0YnVs3RdFQU9IIoIs9PL,spotify:track:5u0YnVs3RdFQU9IIoIs9PL,https://api.spotify.com/v1/tracks/5u0YnVs3RdFQ...,https://api.spotify.com/v1/audio-analysis/5u0Y...,189013.0,4.0,1TtXnWcUs0FCkaZDPGYHdf,Darell,"[latin hip hop, reggaeton, trap latino, urban

In [36]:
features = get_playlist_features(playlists['Pop Rising'])
df_10 = pd.DataFrame(features) 
df_10['playlist_date'] = month_day_year
final_df = final_df.append(df_10)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5hlPrSCx9BvnQ3rCTkblMZ,High,Angel Face,2023-09-22,83,161.567,False,[Stephen Sanchez],0.414,0.678,6.0,-7.826,0.0,0.0355,0.096200,0.000193,0.2740,0.395,176.059,audio_features,5hlPrSCx9BvnQ3rCTkblMZ,spotify:track:5hlPrSCx9BvnQ3rCTkblMZ,https://api.spotify.com/v1/tracks/5hlPrSCx9Bvn...,https://api.spotify.com/v1/audio-analysis/5hlP...,161568.0,4.0,5XKFrudbV4IiuE5WuTPRmT,Stephen Sanchez,[gen z singer-songwriter],1351289,77,37i9dQZF1DWUa8ZRTf

In [37]:
features = get_playlist_features(playlists['MARROW'])
df_11 = pd.DataFrame(features) 
df_11['playlist_date'] = month_day_year
final_df = final_df.append(df_11)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,3tHyJBRSh4B8zwxmDNPVbf,Cartwheels,Cartwheels,2024-04-02,49,124.013,False,[Soft Launch],0.573,0.901,10.0,-2.390,0.0,0.1280,0.023800,0.001250,0.2740,0.697,147.022,audio_features,3tHyJBRSh4B8zwxmDNPVbf,spotify:track:3tHyJBRSh4B8zwxmDNPVbf,https://api.spotify.com/v1/tracks/3tHyJBRSh4B8...,https://api.spotify.com/v1/audio-analysis/3tHy...,124013.0,1.0,4hq1lrK8XlrErNZ3xs3586,Soft Launch,[],2366,30,37i9dQZF1DXcF6B6QPhFDv,MARROW,feel it in your

In [38]:
features = get_playlist_features(playlists['All New Rock'])
df_12 = pd.DataFrame(features) 
df_12['playlist_date'] = month_day_year
final_df = final_df.append(df_12)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,1ZFHhq1dssiLeVujxXOBMM,Dark Empath,First Second,2024-04-19,0,229.240,False,[Pulse Park],0.567,0.803,7.0,-8.396,0.0,0.0271,0.000013,0.036900,0.1320,0.537,109.995,audio_features,1ZFHhq1dssiLeVujxXOBMM,spotify:track:1ZFHhq1dssiLeVujxXOBMM,https://api.spotify.com/v1/tracks/1ZFHhq1dssiL...,https://api.spotify.com/v1/audio-analysis/1ZFH...,229240.0,4.0,2FyCwzesAaA6pyxgbT7H36,Pulse Park,[],532,8,37i9dQZF1DWZryfp6NSvtz,All New Rock,The best new

In [39]:
features = get_playlist_features(playlists['mint'])
df_13 = pd.DataFrame(features) 
df_13['playlist_date'] = month_day_year
final_df = final_df.append(df_13)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0r3nMF80NSIuPIxeod4aoG,Beso (Fruta Fresca),Beso (Fruta Fresca),2023-12-13,73,197.694,False,"[Wakyin, Carlos Vives]",0.778,0.728,10.0,-6.920,0.0,0.1380,0.054000,0.007550,0.0861,0.504,123.968,audio_features,0r3nMF80NSIuPIxeod4aoG,spotify:track:0r3nMF80NSIuPIxeod4aoG,https://api.spotify.com/v1/tracks/0r3nMF80NSIu...,https://api.spotify.com/v1/audio-analysis/0r3n...,197695.0,4.0,2CnJJITnV40TB5UtCRhwIZ,Wakyin,[],16715,55,37i9dQZF1DX4dyzvuaRJ

In [40]:
features = get_playlist_features(playlists['Front Page Indie'])
df_14 = pd.DataFrame(features) 
df_14['playlist_date'] = month_day_year
final_df = final_df.append(df_14)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,039Mnm9c2b2zI5j1iENJ7P,Cosmic Dancer (with Dora Jar),Songs That Aren't Mine,2024-03-08,45,178.584,False,"[Matt Maltese, Dora Jar]",0.411,0.542,8.0,-9.313,1.0,0.0340,0.850000,0.746000,0.1520,0.231,136.726,audio_features,039Mnm9c2b2zI5j1iENJ7P,spotify:track:039Mnm9c2b2zI5j1iENJ7P,https://api.spotify.com/v1/tracks/039Mnm9c2b2z...,https://api.spotify.com/v1/audio-analysis/039M...,178585.0,4.0,12j6dJrPXanCBwY599pZxf,Matt Maltese,"[indie anth

In [41]:
features = get_playlist_features(playlists['Chill Pop'])
df_15 = pd.DataFrame(features) 
df_15['playlist_date'] = month_day_year
final_df = final_df.append(df_15)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7D0RhFcb3CrfPuTJ0obrod,Sparks,Parachutes,2000-07-10,87,227.093,False,[Coldplay],0.371,0.268,1.0,-10.506,1.0,0.0281,0.748000,0.051700,0.1040,0.165,102.617,audio_features,7D0RhFcb3CrfPuTJ0obrod,spotify:track:7D0RhFcb3CrfPuTJ0obrod,https://api.spotify.com/v1/tracks/7D0RhFcb3Crf...,https://api.spotify.com/v1/audio-analysis/7D0R...,227093.0,3.0,4gzpq5DPGxSnKTe4SA8HAU,Coldplay,"[permanent wave, pop]",49959462,88,37i9dQZF1DX0MLFaUdXnjA,Chill P

In [42]:
features = get_playlist_features(playlists['Fresh Finds Country'])
df_16 = pd.DataFrame(features) 
df_16['playlist_date'] = month_day_year
final_df = final_df.append(df_16)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,51nssugo46zxS7DPvOsEcp,Don't Trust That Truck,Don't Trust That Truck,2024-03-15,29,184.599,False,[Heartwreckers],0.422,0.873,8.0,-3.340,1.0,0.0594,0.009530,0.000000,0.1080,0.558,158.208,audio_features,51nssugo46zxS7DPvOsEcp,spotify:track:51nssugo46zxS7DPvOsEcp,https://api.spotify.com/v1/tracks/51nssugo46zx...,https://api.spotify.com/v1/audio-analysis/51ns...,184599.0,4.0,5tLHZ7VEXOqL26DGh3OC3Z,Heartwreckers,[],371,17,37i9dQZF1DWYUfsq4hx

In [43]:
features = get_playlist_features(playlists['RNB X'])
df_17 = pd.DataFrame(features) 
df_17['playlist_date'] = month_day_year
final_df = final_df.append(df_17)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,4hbadZZrbM08KZIQZomWaI,Not An Angel,Not An Angel,2023-12-08,69,185.639,False,[Tems],0.628,0.652,4.0,-6.065,0.0,0.0543,0.010600,0.001570,0.1450,0.552,104.075,audio_features,4hbadZZrbM08KZIQZomWaI,spotify:track:4hbadZZrbM08KZIQZomWaI,https://api.spotify.com/v1/tracks/4hbadZZrbM08...,https://api.spotify.com/v1/audio-analysis/4hba...,185639.0,4.0,687cZJR45JO7jhk1LHIbgq,Tems,"[afro r&b, afrobeats, alte, nigerian pop]",1655151,75,37i9dQZF1DX4

In [44]:
features = get_playlist_features(playlists['African Heat'])
df_18 = pd.DataFrame(features) 
df_18['playlist_date'] = month_day_year
final_df = final_df.append(df_18)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,1NQCinyxiglxekxXVPEhnC,Peacock Revisit (feat. Ice Beats Slide & Sbuda...,SOLACE,2023-08-11,60,326.048,False,"[Uncle Waffles, Ice Beats Slide, Sbuda Maleather]",0.582,0.443,2.0,-11.856,1.0,0.0983,0.007580,0.500000,0.0499,0.260,111.980,audio_features,1NQCinyxiglxekxXVPEhnC,spotify:track:1NQCinyxiglxekxXVPEhnC,https://api.spotify.com/v1/tracks/1NQCinyxiglx...,https://api.spotify.com/v1/audio-analysis/1NQC...,326027.0,4.0,68McnNC9twEtiynOAJ

In [46]:
features = get_playlist_features(playlists['K-Pop Rising'])
df_19 = pd.DataFrame(features) 
df_19['playlist_date'] = month_day_year
final_df = final_df.append(df_19)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,70efTlnBNM8BvfhfPiqOBN,Nobody Knows,Born to be XX,2023-11-08,75,195.960,False,[KISS OF LIFE],0.655,0.646,7.0,-4.322,1.0,0.1160,0.278000,0.000000,0.1560,0.927,90.055,audio_features,70efTlnBNM8BvfhfPiqOBN,spotify:track:70efTlnBNM8BvfhfPiqOBN,https://api.spotify.com/v1/tracks/70efTlnBNM8B...,https://api.spotify.com/v1/audio-analysis/70ef...,195960.0,4.0,4TEK9tIkcoxib4GxT3O4ky,KISS OF LIFE,[k-pop girl group],706423,70,37i9dQZF1DX4FcAKI5Nhzq

In [47]:
features = get_playlist_features(playlists['Yalla'])
df_20 = pd.DataFrame(features) 
df_20['playlist_date'] = month_day_year
final_df = final_df.append(df_20)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,1.0,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515.0,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000.0,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,1.0,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424.0,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,1.0,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246.0,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,1.0,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639.0,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2xxrKfNmahZzrCgR3gfJcN,Befdiki,Befdiki,2023-05-04,56,214.956,False,[AL SHAMI],0.676,0.646,0.0,-7.995,1.0,0.1160,0.105000,0.000006,0.0645,0.473,115.060,audio_features,2xxrKfNmahZzrCgR3gfJcN,spotify:track:2xxrKfNmahZzrCgR3gfJcN,https://api.spotify.com/v1/tracks/2xxrKfNmahZz...,https://api.spotify.com/v1/audio-analysis/2xxr...,214957.0,4.0,597R32EkdYgLMf1tQcHF8k,AL SHAMI,[syrian hip hop],223480,56,37i9dQZF1DX5cO1uP1XC1g,Yalla,The biggest A

In [60]:
features = get_playlist_features(playlists['Dance Rising'])
df_21 = pd.DataFrame(features) 
df_21['playlist_date'] = month_day_year
final_df = final_df.append(df_21)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ' 'Dance Rising']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date,mode,type,id,uri,track_href,analysis_url,duration_ms
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,0.0584,0.035100,0.002590,0.0707,0.912,159.982,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,0.0447,0.065700,0.000032,0.0751,0.287,115.830,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,4KMGgiCvRLHaT5l1L0ACvj,Taking It Back,Taking It Back,2024-02-08,31,218.960,False,[DJ Minx],0.626,0.990,0.0,-6.246,0.0426,0.010700,0.185000,0.1130,0.603,124.968,4.0,4PTQtiKISN5iGNpbRVv02B,DJ Minx,[detroit house],5777,26,37i9dQZF1DX8tZsk68tuDw,Dance Rising,"Tomorrow's <a href=""spotify:genre:edm_dance"">d...",1964139,MTcxMzQ5OTIwMCwwMDAwMDAwMGZiYjNmMDA3OTc2ZjQwMT...,04-20-2024,1.0,audio_features,4KMGgiCvRLHaT5l1L0ACvj,spotify:track:4KMGgiCvRLHaT5l1L0ACvj,https://api.spotify.com/v1/tracks/4KMGgiCvRLHa...,https://api.spotify.com/v1/audio-analysis/4KMG...,218960.0
71,0UcgETwF7Rj8ssMJaqEKvS,Full Speed (feat. Aviella),Full Speed (feat. Aviella),2024-03-15,36,210.400,False,"[CHYL, Aviella]",0.532,0.930,6.0,-4.331,0.1910,0.008320,0.000022,0.1100,0.467,149.986,4.0,15HOfHbNWedCAcJ3Cm1mbc,CHYL,[speed house],18428,39,37i9dQZF1DX8tZsk68tuDw,Dance Rising,"Tomorrow's <a href=""spotify:genre:edm_dance"">d...",1964139,MTcxMzQ5OTIwMCwwMDAwMDAwMGZiYjNmMDA3OTc2ZjQwMT...,04-20-2024,1.0,audio_features,0UcgETwF7Rj8ssMJaqEKvS,spotify:track:0UcgETwF7Rj8ssMJaqEKvS,https://api.spotify.com/v1/tracks/0UcgETwF7Rj8...,https://api.spotify.com/v1/audio-analysis/0Ucg...,210400.0
72,2tseEPCbD2xdVl86U4ZYgX,Talk To Me,Talk To Me,2024-03-19,59,198.437,False,"

In [64]:
features = get_playlist_features(playlists['sad hour'])
df_22 = pd.DataFrame(features) 
df_22['playlist_date'] = month_day_year
final_df = final_df.append(df_22)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ' 'Dance Rising' 'sad hour']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date,mode,type,id,uri,track_href,analysis_url,duration_ms
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,0.0584,0.035100,0.002590,0.0707,0.912,159.982,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,0.0447,0.065700,0.000032,0.0751,0.287,115.830,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2EBtLxQSV8EHpk4gHoMnl3,Older,Older,2024-02-13,68,201.303,False,[Lizzy McAlpine],0.330,0.184,3.0,-10.683,0.0401,0.903000,0.000000,0.1100,0.188,82.895,3.0,1GmsPCcpKgF9OhlNXjOsbS,Lizzy McAlpine,"[alt z, boston folk, indie pop, singer-songwri...",892614,75,37i9dQZF1DWSqBruwoIXkA,sad hour,you don't get to tell me about sad! Cover: Ben...,1757867,MTcxMzUwNjQwMCwwMDAwMDAwMDI4M2I1NjAyNmEzMjNjYm...,04-20-2024,1.0,audio_features,2EBtLxQSV8EHpk4gHoMnl3,spotify:track:2EBtLxQSV8EHpk4gHoMnl3,https://api.spotify.com/v1/tracks/2EBtLxQSV8EH...,https://api.spotify.com/v1/audio-analysis/2EBt...,201304.0
96,0hiHNxqKGf3GZgVTIOAiiK,Someone Like You (feat. Joy Oladokun),I Was / I Am,2021-09-17,68,186.552,False,"[Noah Kahan, Joy Oladokun]",0.614,0.526,5.0,-7.492,0.0603,0.468000,0.000000,0.0939,0.366,177.913,4.0,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,[pov: indie],2644168,87,37i9dQZF1DWSqBruwoIXkA,sad hour,you don't get to tell me about sad! Cover: Ben...,1757867,MTcxMzUwNjQwMCwwMDAwMDAwMDI4M2I1NjAyNmEzMjNjYm...,04-20-2024,1.0,audio_features,0hiHNxqKGf3GZgVTIOAiiK,spotify:track:0hiHNxqKGf3GZgVTIOAiiK,https://api.spotify.com/v1/tracks/0hiHNxqKGf3G...,https://api.spotify.com/v1/audio-analysis/0hiH...,186552.0
97,2At18rPHOuTEjGEppwGV0A,Peter Pan Was Righ

In [66]:
features = get_playlist_features(playlists['Alternative Hip-Hop'])
df_23 = pd.DataFrame(features) 
df_23['playlist_date'] = month_day_year
final_df = final_df.append(df_23)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ' 'Dance Rising' 'sad hour'
 'Alternative Hip-Hop']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date,mode,type,id,uri,track_href,analysis_url,duration_ms
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,0.0584,0.035100,0.002590,0.0707,0.912,159.982,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,0.0447,0.065700,0.000032,0.0751,0.287,115.830,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6ETnEOSqpbICvS58X9f4CK,US,JUMPIN IN / US,2024-03-06,42,148.773,True,[Kenny Mason],0.613,0.674,5.0,-5.008,0.0422,0.006380,0.000000,0.2110,0.154,120.012,4.0,4mwdnO2jZrMmMVrjcHsZBv,Kenny Mason,"[atl hip hop, chill abstract hip hop, indie hi...",170050,60,37i9dQZF1DWTggY0yqBxES,Alternative Hip-Hop,"New, leftfield, and for the real hip-hop stans...",947093,MTcxMzQ5OTIwMCwwMDAwMDAwMGE0ZjU4MTg3ZTE3NTE2OT...,04-20-2024,1.0,audio_features,6ETnEOSqpbICvS58X9f4CK,spotify:track:6ETnEOSqpbICvS58X9f4CK,https://api.spotify.com/v1/tracks/6ETnEOSqpbIC...,https://api.spotify.com/v1/audio-analysis/6ETn...,148773.0
96,3MJhp4duKazKhnp0Of1mom,Don't Check 4 Me (feat. Duckwrth & Saba),Don't Check 4 Me (feat. Duckwrth & Saba),2024-03-29,56,148.846,True,"[Nascent, Duckwrth, Saba]",0.711,0.597,8.0,-7.480,0.2590,0.293000,0.000000,0.1080,0.597,129.645,4.0,5727wMPGcNsG22DBboWpX6,Nascent,"[indie hip hop, indie r&b]",13079,44,37i9dQZF1DWTggY0yqBxES,Alternative Hip-Hop,"New, leftfield, and for the real hip-hop stans...",947093,MTcxMzQ5OTIwMCwwMDAwMDAwMGE0ZjU4MTg3ZTE3NTE2OT...,04-20-2024,1.0,audio_features,3MJhp4duKazKhnp0Of1mom,spotify:track:3MJhp4duKazKhnp0Of1mom,https://api.spotify.com/v1/tracks/3MJhp4duKazK...,https://api.spotify.com/v1/audio-analysi

In [70]:
features = get_playlist_features(playlists['K-Pop ON! (온)'])
df_24 = pd.DataFrame(features) 
df_24['playlist_date'] = month_day_year
final_df = final_df.append(df_24)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ' 'Dance Rising' 'sad hour'
 'Alternative Hip-Hop' 'K-Pop ON! (온)']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date,mode,type,id,uri,track_href,analysis_url,duration_ms
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.9460,11.0,-4.263,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.4990,8.0,-8.472,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.6200,10.0,-5.505,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.4540,2.0,-7.643,0.0584,0.035100,0.002590,0.0707,0.912,159.982,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.6630,5.0,-8.305,0.0447,0.065700,0.000032,0.0751,0.287,115.830,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,3WFTUAPrXvqg3SXC0bsukz,EENIE MEENIE (Feat. Hongjoong of ATEEZ),EENIE MEENIE,2024-03-11,71,170.066,False,"[CHUNG HA, Hongjoong of ATEEZ]",0.819,0.7190,1.0,-4.187,0.1980,0.176000,0.000000,0.0558,0.703,99.892,4.0,2PSJ6YriU7JsFucxACpU7Y,CHUNG HA,[k-pop],2175260,62,37i9dQZF1DX9tPFwDMOaN1,K-Pop ON! (온),Turn on the movement with the latest and great...,5390817,MTcxMzUyMTIxOSwwMDAwMDAwMDk5MDYyNjU5NGIxMjUyMj...,04-20-2024,1.0,audio_features,3WFTUAPrXvqg3SXC0bsukz,spotify:track:3WFTUAPrXvqg3SXC0bsukz,https://api.spotify.com/v1/tracks/3WFTUAPrXvqg...,https://api.spotify.com/v1/audio-analysis/3WFT...,170067.0
46,2t3F6nsQqEw60KT8ma8dm4,Rooftop,Borderline,2024-03-14,64,168.653,False,[YooA],0.852,0.9220,1.0,-3.700,0.1790,0.314000,0.000000,0.5610,0.555,99.970,4.0,4ur1jCwyNlhgd0viJkOtcQ,YooA,[],130642,49,37i9dQZF1DX9tPFwDMOaN1,K-Pop ON! (온),Turn on the movement with the latest and great...,5390817,MTcxMzUyMTIxOSwwMDAwMDAwMDk5MDYyNjU5NGIxMjUyMj...,04-20-2024,0.0,audio_features,2t3F6nsQqEw60KT8ma8dm4,spotify:track:2t3F6nsQqEw60KT8ma8dm4,https://api.spotify.com/v1/tracks/2t3F6nsQqEw6...,https://api.spotify.com/v1/audio-analysis/2t3F...,168653.0
47,5FJqpWkacWPUrXtJ5waI1j,Sugar Rush,Bam Yang Gang,2024-02-13,70,148.729,False,[BIBI],0.874,

In [71]:
features = get_playlist_features(playlists['Hot Hits USA'])
df_25 = pd.DataFrame(features) 
df_25['playlist_date'] = month_day_year
final_df = final_df.append(df_25)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ' 'Dance Rising' 'sad hour'
 'Alternative Hip-Hop' 'K-Pop ON! (온)' 'Hot Hits USA']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date,mode,type,id,uri,track_href,analysis_url,duration_ms
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,0.0584,0.035100,0.002590,0.0707,0.912,159.982,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,0.0447,0.065700,0.000032,0.0751,0.287,115.830,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,2H2ytI7ompfNmRkI2sq8Uk,I LUV IT (feat. Playboi Carti),I LUV IT (feat. Playboi Carti),2024-03-27,87,174.880,True,"[Camila Cabello, Playboi Carti]",0.587,0.779,5.0,-5.764,0.0467,0.014900,0.000394,0.2270,0.228,151.010,4.0,4nDoRrQiYLoBzwC5BhVJzF,Camila Cabello,"[dance pop, pop]",32731995,81,37i9dQZF1DX0kbJZpiYdZl,Hot Hits USA,The hottest tracks in the United States. Cover...,1436884,MTcxMzU0MjgwMSwwMDAwMDAwMGUyMWRlNTEwOWE3MjU1Mm...,04-20-2024,1.0,audio_features,2H2ytI7ompfNmRkI2sq8Uk,spotify:track:2H2ytI7ompfNmRkI2sq8Uk,https://api.spotify.com/v1/tracks/2H2ytI7ompfN...,https://api.spotify.com/v1/audio-analysis/2H2y...,174886.0
46,0eBFgRxyVSeuT4iyrbukdn,Where the Wild Things Are,Gettin' Old,2023-03-24,86,239.533,False,[Luke Combs],0.708,0.731,3.0,-4.872,0.0345,0.018900,0.000004,0.1290,0.379,116.975,4.0,718COspgdWOnwOFpJHRZHS,Luke Combs,"[contemporary country, country]",10334363,85,37i9dQZF1DX0kbJZpiYdZl,Hot Hits USA,The hottest tracks in the United States. Cover...,1436884,MTcxMzU0MjgwMSwwMDAwMDAwMGUyMWRlNTEwOWE3MjU1Mm...,04-20-2024,1.0,audio_features,0eBFgRxyVSeuT4iyrbukdn,spotify:track:0eBFgRxyVSeuT4iyrbukdn,https://api.spotify.com/v1/tracks/0eBFgRxyVSeu...,https://api.spotify.com/v1/audio-analysis/0eBF...,239533.0
47,7CyPw

In [74]:
features = get_playlist_features(playlists["it's alt good"])
df_26 = pd.DataFrame(features) 
df_26['playlist_date'] = month_day_year
final_df = final_df.append(df_26)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ' 'Dance Rising' 'sad hour'
 'Alternative Hip-Hop' 'K-Pop ON! (온)' 'Hot Hits USA' "it's alt good"]


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date,mode,type,id,uri,track_href,analysis_url,duration_ms
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,0.0584,0.035100,0.002590,0.0707,0.912,159.982,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,0.0447,0.065700,0.000032,0.0751,0.287,115.830,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2FhJvFKyZng57iVQmFayVw,Daphne Blue,tryhard,2019-08-23,50,182.840,False,[The Band CAMINO],0.567,0.925,1.0,-2.775,0.0906,0.002390,0.005400,0.3480,0.704,118.006,4.0,6d4jrmreCmsenscuieJERc,The Band CAMINO,"[hopebeat, pov: indie]",387774,59,37i9dQZF1DX2SK4ytI2KAZ,it's alt good,have fun with it.,712765,MTcxMDExMjYyMywwMDAwMDAwMGY3NWEwMjYyYTQ2YWVkM2...,04-20-2024,0.0,audio_features,2FhJvFKyZng57iVQmFayVw,spotify:track:2FhJvFKyZng57iVQmFayVw,https://api.spotify.com/v1/tracks/2FhJvFKyZng5...,https://api.spotify.com/v1/audio-analysis/2FhJ...,182840.0
96,1hf0PdHw0lnVxq5lfzwjSl,strawberry chainsaw,strawberry chainsaw,2022-08-01,51,156.008,False,[JAWNY],0.615,0.879,5.0,-3.977,0.0451,0.234000,0.000023,0.4220,0.627,100.028,4.0,25pd339V2rRJo84USlcSRP,JAWNY,"[bedroom pop, pov: indie]",319405,63,37i9dQZF1DX2SK4ytI2KAZ,it's alt good,have fun with it.,712765,MTcxMDExMjYyMywwMDAwMDAwMGY3NWEwMjYyYTQ2YWVkM2...,04-20-2024,1.0,audio_features,1hf0PdHw0lnVxq5lfzwjSl,spotify:track:1hf0PdHw0lnVxq5lfzwjSl,https://api.spotify.com/v1/tracks/1hf0PdHw0lnV...,https://api.spotify.com/v1/audio-analysis/1hf0...,156008.0
97,2JyoU2NFGjWWTaknPfXeDR,Malibu,Malibu,2022-07-28,50,167.556,True,"[THE DRIVER ERA, Ross Lynch, Rocky]",0.858,0.661,4.0,-6.696,0.0686,0.285

In [77]:
features = get_playlist_features(playlists["Spring Pop 🌸"])
df_27 = pd.DataFrame(features) 
df_27['playlist_date'] = month_day_year
final_df = final_df.append(df_27)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ' 'Dance Rising' 'sad hour'
 'Alternative Hip-Hop' 'K-Pop ON! (온)' 'Hot Hits USA' "it's alt good"
 'Spring Pop 🌸']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date,mode,type,id,uri,track_href,analysis_url,duration_ms
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,0.0584,0.035100,0.002590,0.0707,0.912,159.982,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,0.0447,0.065700,0.000032,0.0751,0.287,115.830,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,7s2kWabRM60W9I61HpKg8C,Solar Power,Solar Power,2021-08-20,65,192.621,False,[Lorde],0.730,0.483,4.0,-8.565,0.0879,0.237000,0.000080,0.3100,0.883,87.996,4.0,163tK9Wjr9P9DmM0AVK7lm,Lorde,"[art pop, metropopolis, nz pop, pop]",9941325,76,37i9dQZF1DX9nkam6FKfgM,Spring Pop 🌸,It’s giving sweatshirt but no coat,68085,MTcxMTQ5MTcxNSwwMDAwMDAwMDY4Zjk3NWIxMjdmZThmZD...,04-20-2024,1.0,audio_features,7s2kWabRM60W9I61HpKg8C,spotify:track:7s2kWabRM60W9I61HpKg8C,https://api.spotify.com/v1/tracks/7s2kWabRM60W...,https://api.spotify.com/v1/audio-analysis/7s2k...,192621.0
56,30VHURM5NOfoZmqp2iKawS,Butterflies,Golden Hour,2018-03-30,70,219.053,False,[Kacey Musgraves],0.689,0.614,4.0,-6.137,0.0306,0.252000,0.000019,0.1780,0.411,141.970,4.0,70kkdajctXSbqSMJbQO424,Kacey Musgraves,"[classic texas country, contemporary country, ...",1388067,80,37i9dQZF1DX9nkam6FKfgM,Spring Pop 🌸,It’s giving sweatshirt but no coat,68085,MTcxMTQ5MTcxNSwwMDAwMDAwMDY4Zjk3NWIxMjdmZThmZD...,04-20-2024,0.0,audio_features,30VHURM5NOfoZmqp2iKawS,spotify:track:30VHURM5NOfoZmqp2iKawS,https://api.spotify.com/v1/tracks/30VHURM5NOfo...,https://api.spotify.com/v1/audio-analysis/30VH...,219053.0
57,6xkryXuiZU360Lngd4sx13,Fantasy,Daydream,1995-10-03,78,243.493,False,[Mariah Care

In [79]:
features = get_playlist_features(playlists["Fuego"])
df_28 = pd.DataFrame(features) 
df_28['playlist_date'] = month_day_year
final_df = final_df.append(df_28)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ' 'Dance Rising' 'sad hour'
 'Alternative Hip-Hop' 'K-Pop ON! (온)' 'Hot Hits USA' "it's alt good"
 'Spring Pop 🌸' 'Fuego']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date,mode,type,id,uri,track_href,analysis_url,duration_ms
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,0.0584,0.035100,0.002590,0.0707,0.912,159.982,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,0.0447,0.065700,0.000032,0.0751,0.287,115.830,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,43PiEypaceH339CSNiggyj,BBY BOO - REMIX,BBY BOO (REMIX),2024-03-27,84,275.862,True,"[iZaak, Jhayco, Anuel AA]",0.703,0.617,5.0,-5.696,0.1510,0.215000,0.000006,0.4330,0.486,173.970,4.0,4s6Xg38sbqh4xpf4OqhVUK,iZaak,"[reggaeton flow, trap latino, urbano latino]",138378,70,37i9dQZF1DX8sljIJzI0oo,Fuego,Feid & Yandel are bringing the heat to Fuego. ...,1880645,MTcxMzQxMjgwMCwwMDAwMDAwMGY5ZmEwYmMwN2FhY2ZjND...,04-20-2024,1.0,audio_features,43PiEypaceH339CSNiggyj,spotify:track:43PiEypaceH339CSNiggyj,https://api.spotify.com/v1/tracks/43PiEypaceH3...,https://api.spotify.com/v1/audio-analysis/43Pi...,275862.0
21,7y8wfdPTJT94sIs6LnGDJC,Trending Remix (with Myke Towers),Trending Remix (with Myke Towers),2023-05-04,80,225.154,True,"[Dei V, Myke Towers]",0.677,0.719,1.0,-3.489,0.1450,0.299000,0.000000,0.0978,0.558,97.948,4.0,2YRyPiW98bpkARAS4B3OQP,Dei V,"[trap latino, urbano latino]",697020,81,37i9dQZF1DX8sljIJzI0oo,Fuego,Feid & Yandel are bringing the heat to Fuego. ...,1880645,MTcxMzQxMjgwMCwwMDAwMDAwMGY5ZmEwYmMwN2FhY2ZjND...,04-20-2024,0.0,audio_features,7y8wfdPTJT94sIs6LnGDJC,spotify:track:7y8wfdPTJT94sIs6LnGDJC,https://api.spotify.com/v1/tracks/7y8wfdPTJT94...,https://api.spotify.com/v1/audio-analysis/7y8w...,225155.0
22,5KZ4EBaAXU

In [82]:
features = get_playlist_features(playlists["Today's Indie Rock"])
df_29 = pd.DataFrame(features) 
df_29['playlist_date'] = month_day_year
final_df = final_df.append(df_29)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ' 'Dance Rising' 'sad hour'
 'Alternative Hip-Hop' 'K-Pop ON! (온)' 'Hot Hits USA' "it's alt good"
 'Spring Pop 🌸' 'Fuego' "Today's Indie Rock"]


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date,mode,type,id,uri,track_href,analysis_url,duration_ms
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,0.0584,0.035100,0.002590,0.0707,0.912,159.982,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,0.0447,0.065700,0.000032,0.0751,0.287,115.830,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7HyJAC5Z0JDS7rq7gbK7EW,Bored,Bored,2024-02-13,46,175.120,False,[Waxahatchee],0.613,0.805,9.0,-3.570,0.0461,0.004670,0.000008,0.1170,0.470,114.987,4.0,5IWCU0V9evBlW4gIeGY4zF,Waxahatchee,"[alabama indie, bubblegrunge, chamber pop, cou...",204664,61,37i9dQZF1DX30HHrCAl4ZG,Today's Indie Rock,Explore all things indie rock right now. Featu...,831656,MTcxMzQ5OTM1NSwwMDAwMDAwMDU4NzczOTNhYThmMmY2MG...,04-20-2024,1.0,audio_features,7HyJAC5Z0JDS7rq7gbK7EW,spotify:track:7HyJAC5Z0JDS7rq7gbK7EW,https://api.spotify.com/v1/tracks/7HyJAC5Z0JDS...,https://api.spotify.com/v1/audio-analysis/7HyJ...,175120.0
96,6LFKTSenkUPdjtoGk5VIN0,Girls That Play,Girls That Play,2024-01-10,48,202.302,False,[Etta Marcus],0.525,0.702,0.0,-5.016,0.0343,0.005050,0.000003,0.5070,0.346,101.972,4.0,2OWXhbFm7SsZN0DXAD7iBg,Etta Marcus,[],30937,44,37i9dQZF1DX30HHrCAl4ZG,Today's Indie Rock,Explore all things indie rock right now. Featu...,831656,MTcxMzQ5OTM1NSwwMDAwMDAwMDU4NzczOTNhYThmMmY2MG...,04-20-2024,1.0,audio_features,6LFKTSenkUPdjtoGk5VIN0,spotify:track:6LFKTSenkUPdjtoGk5VIN0,https://api.spotify.com/v1/tracks/6LFKTSenkUPd...,https://api.spotify.com/v1/audio-analysis/6LFK...,202302.0
97,4lfuRRTLD4UFkSVVArVVaK,Foam,PRATTS & PAIN,2024-02-16,60,185.213,True,[Roye

In [87]:
features = get_playlist_features(playlists["The Scene"])
df_30 = pd.DataFrame(features) 
df_30['playlist_date'] = month_day_year
final_df = final_df.append(df_30)
print(final_df.playlist_name.unique())
final_df

['Top 50 - Global' 'Top 50 - USA' 'Viral 50 - Global' 'Viral 50 - USA'
 'Billboard Hot 100' 'Signed XOXO' 'RapCaviar' 'Viva Latino'
 'Baila Reggaeton' 'Pop Rising' 'MARROW' 'All New Rock' 'mint'
 'Front Page Indie' 'Chill Pop' 'Fresh Finds Country' 'RNB X'
 'African Heat' 'K-Pop Rising' 'Yalla ' 'Dance Rising' 'sad hour'
 'Alternative Hip-Hop' 'K-Pop ON! (온)' 'Hot Hits USA' "it's alt good"
 'Spring Pop 🌸' 'Fuego' "Today's Indie Rock" 'The Scene']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,playlist_date,mode,type,id,uri,track_href,analysis_url,duration_ms
0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,i like the way you kiss me,2024-03-19,99,142.514,False,[Artemas],0.599,0.946,11.0,-4.263,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4.0,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],490543,83,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6XjDF6nds4DE2BBbagZol6,Gata Only,Gata Only,2024-02-02,97,222.000,True,"[FloyyMenor, Cris Mj]",0.791,0.499,8.0,-8.472,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4.0,7CvTknweLr9feJtRGrpDBy,FloyyMenor,[reggaeton chileno],252416,80,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0AjmK0Eai4zGrLaJwPvrDp,Too Sweet,Unreal Unearth: Unheard,2024-03-22,88,251.424,False,[Hozier],0.741,0.620,10.0,-5.505,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4.0,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",6858648,87,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3qhlB30KknSejmIvZZLjOD,End of Beginning,DECIDE,2022-09-16,99,159.245,False,[Djo],0.689,0.454,2.0,-7.643,0.0584,0.035100,0.002590,0.0707,0.912,159.982,4.0,5p9HO3XC5P3BLxJs5Mtrhm,Djo,"[pov: indie, psychedelic pop]",1243997,82,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ZQ1vr10ffzbwIjDCwqm4,we can't be friends (wait for your love),eternal sunshine,2024-03-08,96,228.639,False,[Ariana Grande],0.645,0.663,5.0,-8.305,0.0447,0.065700,0.000032,0.0751,0.287,115.830,4.0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,[pop],95909921,94,37i9dQZEVXbMDoHDwVN2tF,Top 50 - Global,Your daily update of the most played tracks ri...,17077756,NzY2OTA4MDAwLDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMD...,04-20-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3Lu7oymnqlIGUxcHgNShsK,Spitting In The Wind,In Lieu Of Flowers,2024-04-12,42,214.741,True,[Aaron West and The Roaring Twenties],0.329,0.939,4.0,-2.074,0.1200,0.000385,0.000000,0.2330,0.389,179.596,4.0,59cc2f0IvGu6YVEtY4cS0p,Aaron West and The Roaring Twenties,"[alternative emo, anthem emo, emo, philly indi...",40143,45,37i9dQZF1DWZkHEX2YHpDV,The Scene,"pop punk, emo, post-hardcore, alt, & more. Cov...",454714,MTcxMjg5NDQwMCwwMDAwMDAwMDYyYWY5OTI2N2RhZmE3MW...,04-20-2024,1.0,audio_features,3Lu7oymnqlIGUxcHgNShsK,spotify:track:3Lu7oymnqlIGUxcHgNShsK,https://api.spotify.com/v1/tracks/3Lu7oymnqlIG...,https://api.spotify.com/v1/audio-analysis/3Lu7...,214741.0
96,0BI3OuWWFMx0pboFt0YGcC,Hot Mess,Hot Mess,2024-01-09,42,162.741,True,[408],0.455,0.853,9.0,-2.499,0.0442,0.003990,0.000000,0.6000,0.855,185.948,4.0,1m2wYIvVYvhEnvdaOJbIfT,408,[pop punk],51504,47,37i9dQZF1DWZkHEX2YHpDV,The Scene,"pop punk, emo, post-hardcore, alt, & more. Cov...",454714,MTcxMjg5NDQwMCwwMDAwMDAwMDYyYWY5OTI2N2RhZmE3MW...,04-20-2024,1.0,audio_features,0BI3OuWWFMx0pboFt0YGcC,spotify:track:0BI3OuWWFMx0pboFt0YGcC,https://api.spotify.com/v1/tracks/0BI3OuWWFMx0...,https://api.spotify.com/v1/audio-analysis/0BI3...,162742.0
97,7dK3KXph8IKzuMo89sh6Kg,The Storm,The 

In [10]:
features = get_playlist_features(playlists["Feelin' Myself"])
df_31 = pd.DataFrame(features) 
df_31['playlist_date'] = month_day_year
final_df = final_df.append(df_31)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself"]


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.02220,0.000000,0.1350,0.3700,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.00255,0.000000,0.0772,0.4730,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.09830,0.000000,0.0510,0.5360,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.02550,0.000000,0.0835,0.8830,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
5,1ytVc8fTgA7hAOOR0n4EH7,Sunday Service,Sunday Service,2024-02-09,70,166.581,True,[Latto],3MdXrJWsbVzdn6fe5JYkSQ,Latto,[trap queen],1560580,78,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.851,0.764,2.0,-2.876,1.0,0.3650,0.08880,0.000000,0.1630,0.5350,153.019,audio_features,1ytVc8fTgA7hAOOR0n4EH7,spotify:track:1ytVc8fTgA7hAOOR0n4EH7,https://api.spotify.com/v1/tracks/1ytVc8fTgA7h...,https://api.spotify.com/v1/audio-analysis/1ytV...,166581.0,4.0,04-21-2024
6,79AHPJd8upvNxyK2qQcY8J,HISS,HISS,2024-01-26,79,192.818,True,[Megan Thee Stallion],181bsRPaVXVlUKXrxwZfHK,Megan Thee Stallion,"[houston rap, pop, r&b, rap, trap queen]",892363

In [12]:
features = get_playlist_features(playlists["ALT NOW"])
df_32 = pd.DataFrame(features) 
df_32['playlist_date'] = month_day_year
final_df = final_df.append(df_32)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.022200,0.00000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.002550,0.00000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.098300,0.00000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.025500,0.00000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,4bGTNdEbmrDlvqYJKRKJxR,What About The Children,JPEG RAW,2024-03-22,53,279.359,False,"[Gary Clark Jr., Stevie Wonder]",01aC2ikO4Xgb2LUpf9JfKp,Gary Clark Jr.,"[electric blues, modern blues rock, rock, soul...",916092,59,37i9dQZF1DWVqJMsgEN0F4,ALT NOW,Pearl Jam along with the hottest alternative s...,568401,MTcxMzQ5OTIwMCwwMDAwMDAwMGUwYjFiZTg5MTI5NmQ4OT...,0.607,0.862,8.0,-2.654,1.0,0.0546,0.157000,0.00000,0.4070,0.691,98.063,audio_features,4bGTNdEbmrDlvqYJKRKJxR,spotify:track:4bGTNdEbmrDlvqYJKRKJxR,https://api.spotify.com/v1/tracks/4bGTNdEbmrDl...,https://api.spotify.com/v1/audio-a

In [14]:
features = get_playlist_features(playlists["New Joints"])
df_33 = pd.DataFrame(features) 
df_33['playlist_date'] = month_day_year
final_df = final_df.append(df_33)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.355,0.02220,0.000000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.273,0.00255,0.000000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.287,0.09830,0.000000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.303,0.02550,0.000000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,3jq4545Dy2brkCEumotUv3,Checkpoint Charlie,Checkpoint Charlie,2024-04-19,0,184.090,True,[A$AP Twelvyy],0tPjSrb43a58uznKru1k2P,A$AP Twelvyy,[underground hip hop],320898,51,37i9dQZF1DX4SrOBCjlfVi,New Joints,"All the New Joints featuring Tee Grizzley, Nic...",266912,MTcxMzQ5OTIwMCwwMDAwMDAwMGU0Y2QzMDUxODVjMjM1MT...,0.607,0.804,0.0,-5.237,1.0,0.383,0.60800,0.000000,0.0846,0.703,87.390,audio_features,3jq4545Dy2brkCEumotUv3,spotify:track:3jq4545Dy2brkCEumotUv3,https://api.spotify.com/v1/tracks/3jq4545Dy2br...,https://api.spotify.com/v1/audio-analysis/3jq4...,184091.0,4.0,04-21-2024
46,44A7s

In [17]:
features = get_playlist_features(playlists["Soak Up The Sun"])
df_34 = pd.DataFrame(features) 
df_34['playlist_date'] = month_day_year
final_df = final_df.append(df_34)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.022200,0.000000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.002550,0.000000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.098300,0.000000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.025500,0.000000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,3HSIgdZZJGHBV3F3kradtG,Nobody (feat. dodie),Nobody (feat. dodie),2023-02-23,48,206.640,False,"[half•alive, dodie]",7sOR7gk6XUlGnxj3p9F54k,half•alive,"[alt z, pov: indie]",690936,57,37i9dQZF1DX6ALfRKlHn1t,Soak Up The Sun,the soundtrack for your vitamin d fix.,1063783,MTcxMjg4NDM1MSwwMDAwMDAwMGMzYzMwNGI3YjU5OTA3YT...,0.721,0.707,7.0,-7.215,0.0,0.0900,0.309000,0.000000,0.2300,0.652,99.993,audio_features,3HSIgdZZJGHBV3F3kradtG,spotify:track:3HSIgdZZJGHBV3F3kradtG,https://api.spotify.com/v1/tracks/3HSIgdZZJGHB...,https://api.spotify.com/v1/audio-analysis/3HSI...,206640.0,4.0,04-21

In [19]:
features = get_playlist_features(playlists["Hot Country"])
df_35 = pd.DataFrame(features) 
df_35['playlist_date'] = month_day_year
final_df = final_df.append(df_35)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.02220,0.0,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.00255,0.0,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.09830,0.0,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.02550,0.0,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,2z5t8IRRtt5vwkSzP9umJo,your place,shot in the dark,2023-07-21,72,182.086,False,[Ashley Cooke],2qwXeRk8VBAegbUnf3xdyi,Ashley Cooke,"[contemporary country, country pop, modern cou...",111594,57,37i9dQZF1DX1lVhptIYRda,Hot Country,Today's top country hits. Cover: Chris Young,7472584,MTcxMzQ5OTIwMCwwMDAwMDAwMGE3YjQ0MTRjZDk2YzU2NW...,0.448,0.632,5.0,-4.083,1.0,0.0297,0.13400,0.0,0.3170,0.418,76.968,audio_features,2z5t8IRRtt5vwkSzP9umJo,spotify:track:2z5t8IRRtt5vwkSzP9umJo,https://api.spotify.com/v1/tracks/2z5t8IRRtt5v...,https://api.spotify.com/v1/audio-analysis/2z5t...,182086.0,4.0,04-21-2024
46,1YfyfFBZ

In [21]:
features = get_playlist_features(playlists["Alternative Beats"])
df_36 = pd.DataFrame(features) 
df_36['playlist_date'] = month_day_year
final_df = final_df.append(df_36)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.022200,0.000000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.002550,0.000000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.098300,0.000000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.025500,0.000000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,33nj1ZTE6D7or3w3FobgxG,Tramp,Volume II,2022-04-07,32,224.800,False,[Sad Night Dynamite],6ydvoGN30eRfE78bjTANF7,Sad Night Dynamite,[],66136,43,37i9dQZF1DWXMg4uP5o3dm,Alternative Beats,alternative meets electronic Cover: Artemas,478634,MTcxMTU2NzEwNiwwMDAwMDAwMDM2M2IyZjIzZThkNjRkNj...,0.597,0.668,11.0,-6.904,0.0,0.0292,0.229000,0.000048,0.1810,0.469,74.006,audio_features,33nj1ZTE6D7or3w3FobgxG,spotify:track:33nj1ZTE6D7or3w3FobgxG,https://api.spotify.com/v1/tracks/33nj1ZTE6D7o...,https://api.spotify.com/v1/audio-analysis/33nj...,224800.0,4.0,04-21-2024
96,7KDT0me83hS1RnHJ6EeYKS,

In [24]:
features = get_playlist_features(playlists["Homegrown + Heavy"])
df_37 = pd.DataFrame(features) 
df_37['playlist_date'] = month_day_year
final_df = final_df.append(df_37)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats' 'Homegrown + Heavy']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.022200,0.00000,0.1350,0.3700,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.002550,0.00000,0.0772,0.4730,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.098300,0.00000,0.0510,0.5360,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.025500,0.00000,0.0835,0.8830,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2qAC4zr8AlQkQk4kyoTyrf,The End,The End,2023-08-21,13,200.000,True,"[Rivilin, Baby Gos]",3Zx6YOsYHf5uj5xvnNhhgL,Rivilin,[],8732,37,37i9dQZF1DXdO9cVSlXgg6,Homegrown + Heavy,"Local heavy music, right now. Cover: Northlane",40827,MTcxMjg0NDAwMCwwMDAwMDAwMDI1N2I2ZGYwODU5MDYzOT...,0.535,0.864,8.0,-6.542,1.0,0.0665,0.048600,0.00552,0.1890,0.3700,167.990,audio_features,2qAC4zr8AlQkQk4kyoTyrf,spotify:track:2qAC4zr8AlQkQk4kyoTyrf,https://api.spotify.com/v1/tracks/2qAC4zr8AlQk...,https://api.spotify.com/v1/audio-analysis/2qAC...,200000.0,4.0,04-21-2024
81,2xvFlEHaPxVZ2cNflcwQil,Nafanua,

In [27]:
features = get_playlist_features(playlists["New Pop Picks"])
df_38 = pd.DataFrame(features) 
df_38['playlist_date'] = month_day_year
final_df = final_df.append(df_38)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats' 'Homegrown + Heavy' 'New Pop Picks']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.02220,0.000000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.00255,0.000000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.09830,0.000000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.02550,0.000000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4l4q4SvYblrOQecJdKojJp,back to him,he's just not that into you!,2024-04-05,52,164.958,False,[Amelia Moore],61shKG2hWuRTW2PEAPk7fo,Amelia Moore,[],101450,49,37i9dQZF1DX11otjJ7crqp,New Pop Picks,Essential new pop tracks. Cover: EMELINE,76535,MTcxMzQ5OTIwMCwwMDAwMDAwMGM2ZDA3NzhkM2ZiNGViYT...,0.732,0.603,0.0,-5.990,0.0,0.1000,0.05130,0.000013,0.1230,0.710,141.918,audio_features,4l4q4SvYblrOQecJdKojJp,spotify:track:4l4q4SvYblrOQecJdKojJp,https://api.spotify.com/v1/tracks/4l4q4SvYblrO...,https://api.spotify.com/v1/audio-analysis/4l4q...,164959.0,4.0,04-21-2024
96,4X4qJ8mObTUQ7nVAzfemNY

In [30]:
features = get_playlist_features(playlists["Westside Story"])
df_39 = pd.DataFrame(features) 
df_39['playlist_date'] = month_day_year
final_df = final_df.append(df_39)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats' 'Homegrown + Heavy' 'New Pop Picks' 'Westside Story']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.02220,0.000000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.00255,0.000000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.09830,0.000000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.02550,0.000000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,4oPLgP2Mkkpo5e7lU6OnN7,Lamborghini Dreams (feat. 03 Greedo),Another Summer Night,2023-11-24,34,165.957,True,"[Kamaiyah, 03 Greedo]",3XVpDdKav6C6zwlDXPhMEO,Kamaiyah,"[alternative r&b, cali rap, oakland hip hop]",179693,50,37i9dQZF1DWSvKsRPPnv5o,Westside Story,Holdin' it down for the westside. Cover: Mozzy,725045,MTcxMzQ5OTIwMCwwMDAwMDAwMDcxMjBjNzNiM2IyNDJmMW...,0.760,0.460,1.0,-7.671,1.0,0.0962,0.01930,0.000000,0.1100,0.474,93.954,audio_features,4oPLgP2Mkkpo5e7lU6OnN7,spotify:track:4oPLgP2Mkkpo5e7lU6OnN7,https://api.spotify.com/v1/tracks/4oPLgP2Mkkpo...,https://api.spotify.com/v1/

In [32]:
features = get_playlist_features(playlists["Fuego #052 - Regina's Fan Mixtape"])
df_40 = pd.DataFrame(features) 
df_40['playlist_date'] = month_day_year
final_df = final_df.append(df_40)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats' 'Homegrown + Heavy' 'New Pop Picks' 'Westside Story'
 "Fuego #052 - Regina's Fan Mixtape"]


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.02220,0.00000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.00255,0.00000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.09830,0.00000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.02550,0.00000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,2wxduF3ZwN5adsNjqfcysB,GABBANA,GABBANA,2022-03-30,48,171.881,False,[TAICHU],3ou3XMRNmyDSy6gnC1bSgN,TAICHU,[trap argentino],154848,49,37i9dQZF1DWTwf6UlOz1Hs,Fuego #052 - Regina's Fan Mixtape,Featuring Nathy Peluso on Regina's fan curated...,1525,MTcxMDM0ODUzOSwwMDAwMDAwMDdjZGUzZmJhOGNjNzY4NW...,0.771,0.549,10.0,-4.972,0.0,0.0510,0.00133,0.17600,0.0991,0.679,80.995,audio_features,2wxduF3ZwN5adsNjqfcysB,spotify:track:2wxduF3ZwN5adsNjqfcysB,https://api.spotify.com/v1/tracks/2wxduF3ZwN5a...,https://api.spotify.com/v1/audio-analysis/2wxd...,171882.0,4.0,04-21-2024
17,0uaM2q4FQOswkHruAf37vg

In [34]:
features = get_playlist_features(playlists["Gulf & Western"])
df_41 = pd.DataFrame(features) 
df_41['playlist_date'] = month_day_year
final_df = final_df.append(df_41)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats' 'Homegrown + Heavy' 'New Pop Picks' 'Westside Story'
 "Fuego #052 - Regina's Fan Mixtape" 'Gulf & Western']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.02220,0.000000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.00255,0.000000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.09830,0.000000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.02550,0.000000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,4TnUKixNWMfajncgdSwFoi,Amarillo By Morning,Strait From The Heart,1982-01-01,72,173.466,False,[George Strait],5vngPClqofybhPERIqQMYd,George Strait,"[contemporary country, country, country road]",4220891,73,37i9dQZF1DWUPRADzDnbMq,Gulf & Western,"Chill, beachy tunes to soundtrack your summer.",92278,MTcxMDAwNDkzMiwwMDAwMDAwMGExNjk1MDE2NTViMGY1NG...,0.606,0.361,9.0,-16.031,1.0,0.0346,0.26400,0.000034,0.3800,0.621,121.059,audio_features,4TnUKixNWMfajncgdSwFoi,spotify:track:4TnUKixNWMfajncgdSwFoi,https://api.spotify.com/v1/tracks/4TnUKixNWMfa...,https://api.spotify.com/v1/audio-analysi

In [36]:
features = get_playlist_features(playlists["New Noise"])
df_42 = pd.DataFrame(features) 
df_42['playlist_date'] = month_day_year
final_df = final_df.append(df_42)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats' 'Homegrown + Heavy' 'New Pop Picks' 'Westside Story'
 "Fuego #052 - Regina's Fan Mixtape" 'Gulf & Western' 'New Noise']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.022200,0.000000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.002550,0.000000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.098300,0.000000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.025500,0.000000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1qrUGw2NFIJinugQpN4Csk,February,February,2024-02-01,35,232.665,False,[Deux Visages],0aWanrbf83QSmIRYSYdNOR,Deux Visages,[],11975,32,37i9dQZF1DWT2jS7NwYPVI,New Noise,your cheat sheet for all that's on the rise in...,445063,MTcxMzUzODM2MiwwMDAwMDAwMGMyNzRjZWY3YTIwZjQ1Zm...,0.386,0.676,2.0,-7.301,1.0,0.0351,0.000428,0.039100,0.0914,0.117,82.489,audio_features,1qrUGw2NFIJinugQpN4Csk,spotify:track:1qrUGw2NFIJinugQpN4Csk,https://api.spotify.com/v1/tracks/1qrUGw2NFIJi...,https://api.spotify.com/v1/audio-analysis/1qrU...,232666.0,4.0,04-21-2024
96,6b3sztg64sk7iiKzjLGT2G,Pallor Tricks

In [39]:
features = get_playlist_features(playlists["The New Alt"])
df_43 = pd.DataFrame(features) 
df_43['playlist_date'] = month_day_year
final_df = final_df.append(df_43)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats' 'Homegrown + Heavy' 'New Pop Picks' 'Westside Story'
 "Fuego #052 - Regina's Fan Mixtape" 'Gulf & Western' 'New Noise'
 'The New Alt']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.022200,0.000000,0.1350,0.3700,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.002550,0.000000,0.0772,0.4730,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.098300,0.000000,0.0510,0.5360,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.025500,0.000000,0.0835,0.8830,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,537RDcN9XAScJPv9Rn3XDy,What's Love (feat. MUNA),What's Love (feat. MUNA),2024-01-24,52,189.720,False,"[Empress Of, MUNA]",5QuBVnBPEzwYvFrgBbwpmU,Empress Of,"[art pop, electropop, escape room, etherpop, s...",130349,55,37i9dQZF1DX82GYcclJ3Ug,The New Alt,No rules. No boundaries. The best new alternat...,1113534,MTcxMzQ5OTIwMCwwMDAwMDAwMDY3OGNkZmU1YTAyZDA4Mj...,0.474,0.721,4.0,-7.094,1.0,0.0851,0.189000,0.005420,0.6830,0.0719,129.953,audio_features,537RDcN9XAScJPv9Rn3XDy,spotify:track:537RDcN9XAScJPv9Rn3XDy,https://api.spotify.com/v1/tracks/537RDcN9XASc...,https://api.spotif

In [41]:
features = get_playlist_features(playlists["bossa pop"])
df_44 = pd.DataFrame(features) 
df_44['playlist_date'] = month_day_year
final_df = final_df.append(df_43)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats' 'Homegrown + Heavy' 'New Pop Picks' 'Westside Story'
 "Fuego #052 - Regina's Fan Mixtape" 'Gulf & Western' 'New Noise'
 'The New Alt' 'bossa pop']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.02220,0.000000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.00255,0.000000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.09830,0.000000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.02550,0.000000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2yDWhOjZhKZ0XqPXyjNQhx,Lovesick,Glow,2021-03-19,50,253.983,False,[Alice Phoebe Lou],03uMw43UVu9MsQCcHVSGjX,Alice Phoebe Lou,[south african alternative],356381,59,37i9dQZF1DXcUY9O5yRihK,bossa pop,Bossa nova vibes for the hopeless romantic day...,110576,MTcxMjg5NjUyNiwwMDAwMDAwMDY1YTVjM2MyNmM3NmQ5ZT...,0.558,0.508,8.0,-8.398,1.0,0.0308,0.49800,0.000467,0.1540,0.512,155.782,audio_features,2yDWhOjZhKZ0XqPXyjNQhx,spotify:track:2yDWhOjZhKZ0XqPXyjNQhx,https://api.spotify.com/v1/tracks/2yDWhOjZhKZ0...,https://api.spotify.com/v1/audio-analysis/2yDW...,253983.0,4.0,04-21-2024
71,0ypsakEytT

In [43]:
features = get_playlist_features(playlists["CST"])
df_45 = pd.DataFrame(features) 
df_45['playlist_date'] = month_day_year
final_df = final_df.append(df_45)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats' 'Homegrown + Heavy' 'New Pop Picks' 'Westside Story'
 "Fuego #052 - Regina's Fan Mixtape" 'Gulf & Western' 'New Noise'
 'The New Alt' 'bossa pop' 'CST']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.02220,0.000000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.00255,0.000000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.09830,0.000000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.02550,0.000000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,12tJyz2GOa37X25a8GF2hP,Going To The Top,Going To The Top,2024-02-23,54,116.363,True,[DDG],0WK3H9OErSn5zKOkOV5egm,DDG,"[pop rap, trap]",1280858,62,37i9dQZF1DX91gZ5XTbTPm,CST,The Midwest is always on time. Cover: Chief Keef,126225,MTcxMzQ5OTIwMCwwMDAwMDAwMDlmMTM1Y2Y3MWRmNDM2MT...,0.859,0.674,11.0,-8.951,1.0,0.0865,0.00361,0.000004,0.1280,0.236,132.031,audio_features,12tJyz2GOa37X25a8GF2hP,spotify:track:12tJyz2GOa37X25a8GF2hP,https://api.spotify.com/v1/tracks/12tJyz2GOa37...,https://api.spotify.com/v1/audio-analysis/12tJ...,116364.0,4.0,04-21-2024
46,4Aop9fiCaYFPhnlg3dxD2s,THE FALL 

In [46]:
features = get_playlist_features(playlists["Country Rocks"])
df_45 = pd.DataFrame(features) 
df_45['playlist_date'] = month_day_year
final_df = final_df.append(df_45)
print(final_df.playlist_name.unique())
final_df

["Feelin' Myself" 'ALT NOW' 'New Joints' 'Soak Up The Sun' 'Hot Country'
 'Alternative Beats' 'Homegrown + Heavy' 'New Pop Picks' 'Westside Story'
 "Fuego #052 - Regina's Fan Mixtape" 'Gulf & Western' 'New Noise'
 'The New Alt' 'bossa pop' 'CST' 'Country Rocks']


,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,1.0,0.3550,0.02220,0.000000,0.1350,0.370,135.054,audio_features,7mOC6phfrfz3jfigg1aGsE,spotify:track:7mOC6phfrfz3jfigg1aGsE,https://api.spotify.com/v1/tracks/7mOC6phfrfz3...,https://api.spotify.com/v1/audio-analysis/7mOC...,156444.0,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,1.0,0.2730,0.00255,0.000000,0.0772,0.473,139.166,audio_features,4XqBLxDaEdlqkHlSlb1Rzq,spotify:track:4XqBLxDaEdlqkHlSlb1Rzq,https://api.spotify.com/v1/tracks/4XqBLxDaEdlq...,https://api.spotify.com/v1/audio-analysis/4XqB...,158727.0,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,1.0,0.2870,0.09830,0.000000,0.0510,0.536,145.031,audio_features,2MjXWroB9wlTG2kqv3avfS,spotify:track:2MjXWroB9wlTG2kqv3avfS,https://api.spotify.com/v1/tracks/2MjXWroB9wlT...,https://api.spotify.com/v1/audio-analysis/2MjX...,148552.0,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,1.0,0.3030,0.02550,0.000000,0.0835,0.883,152.046,audio_features,3Vlt0DKYBK0h3Vf92nywhp,spotify:track:3Vlt0DKYBK0h3Vf92nywhp,https://api.spotify.com/v1/tracks/3Vlt0DKYBK0h...,https://api.spotify.com/v1/audio-analysis/3Vlt...,169067.0,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,71Q2un1xXRayfGf0YIHH6j,Break Before I Bend,Break Before I Bend,2023-07-28,40,189.318,False,[Joe Peters],6J1kpiqZmCZQtYB7NJLqVn,Joe Peters,[texas country],1743,28,37i9dQZF1DWWH0izG4erma,Country Rocks,Country Music that rocks. Cover: Ole 60,592494,MTcxMjkzMzE4MywwMDAwMDAwMDM1OTU5N2JkZjA4ZTIzZG...,0.527,0.858,11.0,-2.233,1.0,0.0312,0.00872,0.000000,0.3490,0.509,93.025,audio_features,71Q2un1xXRayfGf0YIHH6j,spotify:track:71Q2un1xXRayfGf0YIHH6j,https://api.spotify.com/v1/tracks/71Q2un1xXRay...,https://api.spotify.com/v1/audio-analysis/71Q2...,189319.0,4.0,04-21-2024
96,2SncBUzb3UsejCoo

In [49]:
features = get_playlist_features(playlists["Country Rap"])
df_46 = pd.DataFrame(features) 
df_46['playlist_date'] = month_day_year
final_df = final_df.append(df_46)
print(final_df.playlist_name.unique())
final_df

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=5ImqyYZ3zgraskdflX6Cdc:
 Max Retries, reason: too many 429 error responses

## Review `final_df` before exporting

In [50]:
### Final changes to final_df before exporting
#final_df['mode'] = final_df['mode'].replace({0: 'minor', 1: 'major'})
final_df = final_df.drop(['mode', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms'], axis=1)

In [51]:
### Show final_df before exporting
final_df

,track_id,track_name,track_album,track_release_date,track_popularity,track_duration_sec,track_explicit,track_all_artists,artist_id,artist_name,artist_genres,artist_followers,artist_popularity,playlist_id,playlist_name,playlist_description,playlist_followers,playlist_snapshot_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,playlist_date
0,2oTFuT1tKLHlTJsnBSJJiN,"FTCU (feat. Travis Scott, Chris Brown & Sexyy ...",FTCU (SLEEZEMIX),2024-04-19,0,239.667,True,"[Nicki Minaj, Travis Scott, Chris Brown, Sexyy...",0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,"[hip pop, pop, queens hip hop, rap]",30886319,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-21-2024
1,7mOC6phfrfz3jfigg1aGsE,Wanna Be (with Megan Thee Stallion),Ehhthang Ehhthang,2024-04-05,79,156.444,True,"[GloRilla, Megan Thee Stallion]",2qoQgPAilErOKCwE2Y8wOG,GloRilla,[southern hip hop],645278,73,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.918,0.616,1.0,-5.202,0.3550,0.02220,0.000000,0.1350,0.370,135.054,4.0,04-21-2024
2,4XqBLxDaEdlqkHlSlb1Rzq,Enough (Miami),Enough (Miami),2024-03-15,72,158.727,True,[Cardi B],4kYSro6naA4h99UJvo89HB,Cardi B,"[pop, rap]",23606695,81,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.926,0.791,5.0,-4.553,0.2730,0.00255,0.000000,0.0772,0.473,139.166,5.0,04-21-2024
3,2MjXWroB9wlTG2kqv3avfS,Get It Sexyy,Get It Sexyy,2024-03-15,84,148.551,True,[Sexyy Red],3DbwFQlvLxRSi2uX8mf81A,Sexyy Red,[trap queen],788904,77,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.806,0.737,0.0,-4.709,0.2870,0.09830,0.000000,0.0510,0.536,145.031,4.0,04-21-2024
4,3Vlt0DKYBK0h3Vf92nywhp,OKLOSER,Scarlet 2 CLAUDE,2024-04-05,78,169.066,True,[Doja Cat],5cj0lLjcoR7YOSnhnX0Po5,Doja Cat,"[dance pop, pop]",29478325,87,37i9dQZF1DX6GwdWRQMQpq,Feelin' Myself,The hip-hop playlist that's a whole mood. Art ...,3112899,MTcxMzQ5OTIwMCwwMDAwMDAwMGMzNTcwOGMzOGE4MDk0ZT...,0.898,0.674,1.0,-8.373,0.3030,0.02550,0.000000,0.0835,0.883,152.046,4.0,04-21-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,71Q2un1xXRayfGf0YIHH6j,Break Before I Bend,Break Before I Bend,2023-07-28,40,189.318,False,[Joe Peters],6J1kpiqZmCZQtYB7NJLqVn,Joe Peters,[texas country],1743,28,37i9dQZF1DWWH0izG4erma,Country Rocks,Country Music that rocks. Cover: Ole 60,592494,MTcxMjkzMzE4MywwMDAwMDAwMDM1OTU5N2JkZjA4ZTIzZG...,0.527,0.858,11.0,-2.233,0.0312,0.00872,0.000000,0.3490,0.509,93.025,4.0,04-21-2024
96,2SncBUzb3UsejCoo4NHhtk,What Do I Know (feat. HARDY),What Do I Know (feat. HARDY),2021-03-05,47,208.418,False,"[Robert Counts, HARDY]",5pKy8QL8CUEtkvOI6dpUm9,Robert Counts,[country pop],6137,32,37i9dQZF1DWWH0izG4erma,Country Rocks,Country Music that rocks. Cover: Ole 60,592494,MTcxMjkzMzE4MywwMDAwMDAwMDM1OTU5N2JkZjA4ZTIzZG...,0.533,0.589,0.0,-6.140,0.0299,0.04050,0.000000,0.1110,0.351,140.049,4.0,04-21-2024
97,4dmNC1afkVZKr5RfHkaj0E,She's No Good for Me,She's No Good for Me,2023-01-20,41,205.651,False,[Meghan Patrick],5bQxLohAAiOat0EMYuHjzJ,Meghan Patrick,"[canadian contemporary country, country pop]",49626,39,37i9dQZF1DWWH0izG4erma,Country Rocks,Country Music that rocks. Cover: Ole 60,592494,MTcxMjkzMzE4MywwMDAwMDAwMDM1OTU5N2JkZjA4ZTIzZG...,0.388,0.812,3.0,-4.291,0.0419,0.01090,0.000002,0.3430,0.321,168.102,4.0,04-21-2024
98,680abMjLEy7wOLgR7zaQv4,World On Fire,World On Fire,2023-05-11,47,261.269,False,[Dolly Parton],32vWCbZh0xZ4o9gkz4PsEU,Dolly Parton,"[classic country pop, country, country dawn]",2439101,75,37i9dQZF1DWWH0izG4erma,Country Rocks,Cou

## Export `final_df`

In [52]:
### Save the DataFrame as a CSV file to directory

# Get current date and time
current_datetime = datetime.now()

# Format the date and time to include in the file name
timestamp = current_datetime.strftime("%Y%m%d_%H%M%S")

# Construct the file name with timestamp
file_name = f'bulk_data_{timestamp}.csv'

# Specify the path to the 'data' subfolder relative to the current directory
data_folder = os.path.join('data - Sprint 02')

# Ensure that the 'data' subfolder exists, create it if it doesn't
os.makedirs(data_folder, exist_ok=True)

# Construct the file path within the 'data' subfolder
file_path = os.path.join(data_folder, file_name)

# Save the DataFrame as a CSV file to the 'data' subfolder
final_df.to_csv(file_path, index=False)

# Print confirmation message
print(f"DataFrame saved as CSV file: {file_path}")

DataFrame saved as CSV file: data - Sprint 02/bulk_data_20240421_024706.csv
